In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays 
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [2]:
Params = get_default_params_PLD()
# Params = get_Wang_Qi_Shen_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
norm_bounds = Params["norm_bounds"]
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_para_Input = Params["coef_this"]; # coefficient for data generation
# coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 0.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = 0.0, r_ub = 0.1)

In [3]:
S_train_list = Params["S_train_all"] # training data size
is_ridge = Params["is_ridge"] # whether to use ridge regression
S_train_list = [500]
lambda_all = [0.005,0.0075,0.01]
gamma_list = [0.0,0.05,0.1,0.2,0.4,0.6,0.8,1.0]

8-element Vector{Float64}:
 0.0
 0.05
 0.1
 0.2
 0.4
 0.6
 0.8
 1.0

In [4]:
Random.seed!(seed)
is_Wang_Qi_Shen = true;
is_same_util_para = true;
if is_Wang_Qi_Shen
    project_dir = "Estimation_Wang_Qi_Shen_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
else
    project_dir = "Test_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
end
if is_same_util_para
    println("Generate data with the same utility parameters for all instances.")
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_para_Input);
    project_dir = string(project_dir, "_Same_Util_Para/")
else
    println("Generate data with different utility parameters for all instances.")
    project_dir = string(project_dir, "_Diff_Util_Para/")
end
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")

data_dir = string(data_dir,project_dir)
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)
save(string(data_dir, "Params.jld2"), Params);

Generate data with the same utility parameters for all instances.
Data directory: /Users/zhangxun/Codes/Data/Product_Line_Design/Estimation_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/


In [5]:
function compute_w(params,z_input)
    alpha0 = params.alpha0
    alpha = params.alpha
    beta = params.beta
    A = params.A
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [6]:
function Estimation_Process(S_train_list,lambda_all,instances)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            if is_same_util_para
                Input_Data_this = Generate_Data_this_Same_Para(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
            else
                Input_Data_this = Generate_Data_this(N_x,N_u,N_nonzero,S_train,S_test,m,coef_para_Input)
            end
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            nu0_true,nu_true = compute_w(theta_true,Z_test[1,:])  
            nu_all_true = [nu0_true;nu_true]

            # ******** Estimation *************
            for lambda in lambda_all
                theta_hat = Estimation_This(N_Max,N_x,N_u,Y_train,X_train,Z_train, Asorrtment_train,is_ridge, lambda)
                nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  
                nu_all_hat = [nu0_hat;nu_hat]

                Input_Data_this["theta_hat_lambda=$(lambda)"] = theta_hat
                Input_Data_this["nu_hat_lambda=$(lambda)"] = nu_all_hat
            end

            Input_Data_this["nu_true"] = nu_all_true
            Input_Data["ins=$(ins)"] = Input_Data_this
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "Input_Data_S=$(S_train).jld2"), Input_Data);
    end
end

Estimation_Process (generic function with 1 method)

In [7]:
function ETO_Optimization_Process(S_train_list,lambda_all,instances)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data_Size = load(string(data_dir, "Input_Data_S=$(S_train).jld2"));

        RST_True_All = Dict()
        RST_ETO_All = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Input_Data_Size["ins=$(ins)"]
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);

            theta_true = Input_Data_this["theta_true"]
            nu_all_true = Input_Data_this["nu_true"]
            Z_test = Input_Data_this["Z_test"]

            # ******** True Model *************
            theta_Input = theta_true
            RST_True,status_True = solve_ETO_This(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
            # println("Oracle: status = ",status_True,",obj=",RST_True["obj"][1])
            if status_True != "OPTIMAL"
                println("Warning: The true model did not reach optimality")
                continue
            end
            RST_True_All["ins=$(ins)"] = RST_True

            # ******** ETO Model *************
            for lambda in lambda_all
                theta_hat = Input_Data_this["theta_hat_lambda=$lambda"]
                nu_all_hat = Input_Data_this["nu_hat_lambda=$lambda"]
                RST_ETO,status_ETO = solve_ETO_This(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
                RST_ETO_All["ins=$(ins)_lambda=$(lambda)"] = RST_ETO
            end
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "RST_True_S=$(S_train).jld2"), RST_True_All);
        save(string(data_dir, "RST_ETO_S=$(S_train).jld2"), RST_ETO_All);
    end
end

ETO_Optimization_Process (generic function with 1 method)

In [8]:
# Estimation_Process(S_train_list,lambda_all,instances)

In [9]:
# ETO_Optimization_Process(S_train_list,lambda_all,instances)

In [10]:
# profit_ETO_all = Dict()
# profit_True_all = Dict()
# for S_train in S_train_list
#     RST_True_All = load(string(data_dir, "RST_True_S=$(S_train).jld2"));
#     RST_ETO_All = load(string(data_dir, "RST_ETO_S=$(S_train).jld2"))

#     profit_True_all["S=$(S_train)"] = [RST_True_All["ins=$(ins)"]["profit"][1] for ins in 1:instances];
#     for lambda in lambda_all
#         profit_ETO_all["S=$(S_train)_lambda=$(lambda)"] = [RST_ETO_All["ins=$(ins)_lambda=$(lambda)"]["profit"][1] for ins in 1:instances];
#     end
# end

In [11]:
# profit_True_Avg = [mean(profit_True_all["S=$(S_train)"]) for S_train in S_train_list]
# profit_ETO_Avg = [[mean(profit_ETO_all["S=$(S_train)_lambda=$(lambda)"]) for lambda in lambda_all] for S_train in S_train_list];

In [12]:
# S_index = 1
# S_train = S_train_list[S_index]
# plot(lambda_all, profit_ETO_Avg[S_index]./profit_True_Avg[S_index],marker=:o,label="S=$S_train")
# for S_index in 2:length(S_train_list)
#     S_train = S_train_list[S_index]
#     plot!(lambda_all, profit_ETO_Avg[S_index]./profit_True_Avg[S_index],marker=:o,label="S=$S_train")
# end
# display(current())

### Model Misspecification

In [13]:
function Estimation_Process_Model_Mis(S_train_list,lambda_all,instances,N_u_list)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data_Size = load(string(data_dir, "Input_Data_S=$(S_train).jld2"));
        Input_Data_Model_Mis = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Input_Data_Size["ins=$(ins)"]
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            
            Input_Data_Model_Mis_this = Dict()
            for lambda in lambda_all
                for N_u_input in N_u_list
                    Z_train_input = Z_train[:,1:N_u_input];
                    Z_test_input = Z_test[:,1:N_u_input];                
                    theta_hat = Estimation_This(N_Max,N_x,N_u_input,Y_train,X_train,Z_train_input, Asorrtment_train,is_ridge, lambda)
                    nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,1:N_u_input])
                    nu_all_hat = [nu0_hat;nu_hat]
                    # println("nu_hat = ",round.(nu_all_hat,digits=2))
                    Input_Data_Model_Mis_this["theta_hat_lambda_$(lambda)_N_u=$(N_u_input)"] = theta_hat
                    Input_Data_Model_Mis_this["nu_hat_lambda=$(lambda)_N_u=$(N_u_input)"] = nu_all_hat
                end
            end
            Input_Data_Model_Mis["ins=$ins"] = Input_Data_Model_Mis_this
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "Input_Data_Model_Mis_S=$(S_train).jld2"), Input_Data_Model_Mis);
    end
end

Estimation_Process_Model_Mis (generic function with 1 method)

In [14]:
function solve_ETO_Omitted_Var(S_test,N,N_x,N_u_input,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
    r0 = r_params.r0; r = r_params.r;

    RST_Dict = Dict()
    obj_list = zeros(S_test); x_list = zeros(S_test,N,N_x); time_list = zeros(S_test); profit_list = zeros(S_test); status_list = Vector{String}(undef, S_test)
    status_all = "OPTIMAL"
    for i in 1:S_test
        z_input = Z_test[i,1:N_u_input];
        nu0, nu = compute_w(theta_Input,z_input)
        # println("nu = ",round.(nu,digits=4))
        obj_list[i], x_list[i,:,:], time_list[i],status_list[i] = ETO_PLD(N,N_x,nu0, nu, r0, r,c_l,d_r,rev_gap,num_c,Time_Limit);
        
        if status_list[i] != "OPTIMAL"
            status_all = status_list[i]
            println("Warning: The optimization for test instance $i did not reach optimality. Status: ", status_list[i])
            break
        end
        profit_list[i] = calculate_profit(theta_true, r0, r, x_list[i,:,:], Z_test[i,:])                       
    end
    RST_Dict["obj"] = obj_list
    RST_Dict["X"] = x_list
    RST_Dict["time"] = time_list
    RST_Dict["profit"] = profit_list
    RST_Dict["status"] = status_list
    return RST_Dict,status_all
end

solve_ETO_Omitted_Var (generic function with 1 method)

In [15]:
function Optimization_Process_Model_Mis(S_train_list,lambda_all,instances,N_u_list)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data = load(string(data_dir, "Input_Data_S=$(S_train).jld2"));
        Input_Data_Model_Mis = load(string(data_dir, "Input_Data_Model_Mis_S=$(S_train).jld2"));

        RST_True_Mis_All = Dict()
        RST_ETO_Mis_All = Dict()
        RST_RO_Mis_All = Dict()
        ins = 1
        while ins <= instances

            # ******** Data generation *************
            Input_Data_this = Input_Data["ins=$ins"]
            Input_Data_Model_Mis_this = Input_Data_Model_Mis["ins=$ins"]
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            
            # ******** True Model *************
            theta_Input = theta_true
            RST_True,status_True = solve_ETO_This(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
            # println("Oracle: status = ",status_True,",obj=",RST_True["obj"][1])
            if status_True != "OPTIMAL"
                println("Warning: The true model did not reach optimality")
                continue
            end
            RST_True_Mis_All["ins=$(ins)"] = RST_True

            for lambda in lambda_all
                for N_u_input in N_u_list
                    theta_Input = Input_Data_Model_Mis_this["theta_hat_lambda_$(lambda)_N_u=$(N_u_input)"]
                    RST_ETO,status_ETO = solve_ETO_Omitted_Var(S_test,N,N_x,N_u_input,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
                    RST_ETO_Mis_All["ins=$(ins)_lambda=$(lambda)_N_u=$N_u_input"] = RST_ETO
                    println("ins=$(ins)_lambda=$(lambda)_N_u=$(N_u_input) profit = ",RST_ETO["profit"][1])
                end
                println("--------------------")
            end

            # # # ******** RO Model *************
            # RST_RO_this = Dict()
            # gamma = gamma_list[1]
            # RST_RO,status_RO = solve_RO_this(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub)
            # # println("gamma = $gamma, RO: status = ",status_RO,",obj=",RST_RO["obj"][1])
            # if status_RO != "OPTIMAL"
            #     println("Warning: The RO model did not reach optimality")
            #     continue
            # end
            # ratio = abs(RST_RO["obj"][1] - RST_ETO["obj"][1])/abs(RST_ETO["obj"][1])
            # # ratio = abs(RST_RO["profit"][1] - RST_ETO["profit"][1])/abs(RST_ETO["profit"][1])
            # if ratio > 1e-3
            #     println("Warning: The RO obj is not equivalent to ETO obj: ETO_Obj=",RST_ETO["obj"][1],",RO_Obj=",RST_RO["obj"][1])
            #     continue
            # end
            # RST_RO_this[string("gamma=",gamma)] = RST_RO

            # for g_index in 2:length(gamma_list)
            #     gamma = gamma_list[g_index]
            #     RST_RO,status_RO = solve_RO_this(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub)
            #     # println("gamma = $gamma, RO: status = ",status_RO,",obj=",RST_RO["obj"][1])
            #     RST_RO_this[string("gamma=",gamma)] = RST_RO
            # end
            # RST_RO_All["ins=$(ins)"] = RST_RO_this

            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        # save(string(data_dir, "Input_Data_S=$(S_train)_lambda=$(lambda).jld2"), Input_Data);
        save(string(data_dir, "RST_True_Mis_S=$(S_train).jld2"), RST_True_Mis_All);
        save(string(data_dir, "RST_ETO_Mis_S=$(S_train).jld2"), RST_ETO_Mis_All);
        save(string(data_dir, "RST_RO_Mis_S=$(S_train).jld2"), RST_RO_Mis_All);
    end
end

Optimization_Process_Model_Mis (generic function with 1 method)

In [16]:
# N_u_list = [0]
# Estimation_Process_Model_Mis(S_train_list,lambda_all,instances,N_u_list)

In [17]:
# Optimization_Process_Model_Mis(S_train_list,lambda_all,instances,N_u_list)

In [18]:
# profit_ETO_all = Dict()
# profit_True_all = Dict()
# for S_train in S_train_list
#     RST_True_All = load(string(data_dir, "RST_True_Mis_S=$(S_train).jld2"));
#     RST_ETO_All = load(string(data_dir, "RST_ETO_Mis_S=$(S_train).jld2"))

#     profit_True_all["S=$(S_train)"] = [RST_True_All["ins=$(ins)"]["profit"][1] for ins in 1:instances];
#     for lambda in lambda_all
#         profit_ETO_all["S=$(S_train)_lambda=$(lambda)"] = [RST_ETO_All["ins=$(ins)_lambda=$(lambda)"]["profit"][1] for ins in 1:instances];
#     end
# end

In [19]:
# profit_True_Avg = [mean(profit_True_all["S=$(S_train)"]) for S_train in S_train_list]
# profit_ETO_Avg = [[mean(profit_ETO_all["S=$(S_train)_lambda=$(lambda)"]) for lambda in lambda_all] for S_train in S_train_list];

In [20]:
# S_index = 1
# S_train = S_train_list[S_index]
# plot(lambda_all, profit_ETO_Avg[S_index]./profit_True_Avg[S_index],marker=:o,label="S=$S_train")
# for S_index in 2:length(S_train_list)
#     S_train = S_train_list[S_index]
#     plot!(lambda_all, profit_ETO_Avg[S_index]./profit_True_Avg[S_index],marker=:o,label="S=$S_train")
# end
# display(current())

### Sin utility function

In [21]:
Random.seed!(seed)
is_Wang_Qi_Shen = true;
is_same_util_para = true;
if is_Wang_Qi_Shen
    project_dir = "Sin_Wang_Qi_Shen_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
else
    project_dir = "Test_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
end
if is_same_util_para
    println("Generate data with the same utility parameters for all instances.")
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_para_Input);
    project_dir = string(project_dir, "_Same_Util_Para/")
else
    println("Generate data with different utility parameters for all instances.")
    project_dir = string(project_dir, "_Diff_Util_Para/")
end
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")

data_dir = string(data_dir,project_dir)
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)
save(string(data_dir, "Params.jld2"), Params);

Generate data with the same utility parameters for all instances.
Data directory: /Users/zhangxun/Codes/Data/Product_Line_Design/Sin_Wang_Qi_Shen_N=3_N_x=8_N_u=10_N_nonzero=20_dr=2.0_seed=2_Same_Util_Para/


In [22]:
function Generate_Wang_Qi_Max_True_Data_Sin(d, p, n, m,theta_true)

    # 初始化存储
    X = Vector{Matrix{Float64}}(undef, n); # n 个 m×d 矩阵
    Z = Matrix{Float64}(undef, n, p);      # n×p 矩阵
    Y = Vector{Int}(undef, n);             # n 维向量

    for i in 1:n
        z_i = rand(Uniform(-0.1, 0.1), p)
        Z[i, :] = z_i

        X_i = zeros(m, d)
        for j in 1:m
            # X_i[j, :] = rand(Uniform(0.0, 1.0), d)
            X_i[j, :] = rand(0.0:1.0, d)
        end
        X[i] = X_i

        # --- 计算选择概率 ---
        # 根据公式 (2.1): Pz(x; θ*) = exp(Uz(x)) / (V0 + exp(Uz(x)))
        # 论文中 V0 = exp(U0) = 1 (默认选项效用权重归一化为1)
        V0 = 1.0
        utilities = zeros(m)
        for j in 1:m
            x_ij = X_i[j, :] # 第 j 个产品的设计
            # 计算效用 Uz(x_ij) = α₀* + <α*, x_ij> + <β*, z_i> + x_ij^T * A* * z_i
            utility = theta_true.alpha0 +
                        dot(theta_true.alpha, x_ij) +
                        dot(theta_true.beta, z_i) +
                        dot(x_ij, theta_true.A * z_i) # x_ij^T * A* * z_i
                        
            utilities[j] = sin(utility)
        end

        # 计算分子 exp(Uz(x_ij))
        exp_utilities = exp.(utilities)
        # 计算分母 (V0 + sum(exp(Uz(x_il))))
        denominator = V0 + sum(exp_utilities)

        # 计算选择每个产品 j 的概率
        prob_choose_product = exp_utilities ./ denominator
        # 计算选择默认选项 (索引 0) 的概率
        prob_choose_default = V0 / denominator

        # 构建完整的概率向量 [P(选择默认), P(选择产品1), ..., P(选择产品m_actual)]
        choice_probs = vcat(prob_choose_default, prob_choose_product)
        # println("Choice probabilities: ", choice_probs)
        # --- 进行多项抽样得到选择结果 y_i ---
        # 使用 StatsBase 的 sample 函数
        # 选择结果: 0 表示默认选项, 1 表示第一个产品, ..., m_actual 表示第 m_actual 个产品
        y_i = sample(0:m, Weights(choice_probs))
        Y[i] = y_i
    end
    return X,Y,Z
end

Generate_Wang_Qi_Max_True_Data_Sin (generic function with 1 method)

In [23]:
function Generate_Data_this_Same_Para_Sin(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed)
    theta_true = theta_true_Fixed
    r_params = r_params_Fixed
    X_train,Y_train,Z_train = Generate_Wang_Qi_Max_True_Data_Sin(N_x, N_u, S_train, N_Max,theta_true);
    X_test,Y_test,Z_test = Generate_Wang_Qi_Max_True_Data_Sin(N_x, N_u, S_test, N_Max,theta_true);
    Input_Data = Dict()
    Input_Data["theta_true"] = theta_true
    Input_Data["r_params"] = r_params
    Input_Data["X_train"] = X_train
    Input_Data["Y_train"] = Y_train
    Input_Data["Z_train"] = Z_train
    Input_Data["X_test"] = X_test
    Input_Data["Y_test"] = Y_test
    Input_Data["Z_test"] = Z_test

    asorrtment_train = Array{Vector{Int64}}(undef,S_train)
    for s in 1:S_train
        asorrtment_train[s] = collect(1:N_Max)
    end
    Input_Data["asorrtment_train"] = asorrtment_train
    return Input_Data
end

Generate_Data_this_Same_Para_Sin (generic function with 1 method)

In [24]:
function Estimation_Process_Sin(S_train_list,lambda_all,instances)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Generate_Data_this_Same_Para_Sin(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            nu0_true,nu_true = compute_w(theta_true,Z_test[1,:])  
            nu_all_true = [nu0_true;nu_true]

            # ******** Estimation *************
            for lambda in lambda_all
                theta_hat = Estimation_This(N_Max,N_x,N_u,Y_train,X_train,Z_train, Asorrtment_train,is_ridge, lambda)
                nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  
                nu_all_hat = [nu0_hat;nu_hat]
                Input_Data_this["theta_hat_lambda=$(lambda)"] = theta_hat
                Input_Data_this["nu_hat_lambda=$(lambda)"] = nu_all_hat
            end
            Input_Data_this["nu_true"] = nu_all_true
            Input_Data["ins=$(ins)"] = Input_Data_this
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "Input_Data_Sin_S=$(S_train).jld2"), Input_Data);
    end
end

Estimation_Process_Sin (generic function with 1 method)

In [25]:
function calculate_profit_Sin(params, r0, r, X_val, z_input)
    alp0 = params.alpha0
    alp = params.alpha
    beta = params.beta
    A = params.A

    V0 = 1.0
    utilities = zeros(N)
    for j in 1:N
        x_ij = X_val[j, :] # 第 j 个产品的设计
        # 计算效用 Uz(x_ij) = α₀* + <α*, x_ij> + <β*, z_i> + x_ij^T * A* * z_i
        utility = alp0 + dot(alp, x_ij) + dot(beta, z_input) + dot(x_ij, A * z_input) # x_ij^T * A* * z_i
        utilities[j] = sin(utility)
    end

    exp_utilities = exp.(utilities)
    # 计算分母 (V0 + sum(exp(Uz(x_il))))
    denominator = V0 + sum(exp_utilities)

    # 计算选择每个产品 j 的概率
    prob_choose_product = exp_utilities ./ denominator
    # 计算选择默认选项 (索引 0) 的概率
    prob_choose_default = V0 / denominator

    profits = r0 .+ X_val * r 
    total_profit = profits' * prob_choose_product
    return total_profit
end

calculate_profit_Sin (generic function with 1 method)

In [26]:
function solve_ETO_This_Sin(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
    r0 = r_params.r0; r = r_params.r;

    RST_Dict = Dict()
    obj_list = zeros(S_test); x_list = zeros(S_test,N,N_x); time_list = zeros(S_test); profit_list = zeros(S_test); status_list = Vector{String}(undef, S_test)
    status_all = "OPTIMAL"
    for i in 1:S_test
        z_input = Z_test[i,:];
        nu0, nu = compute_w(theta_Input,z_input)
        obj_list[i], x_list[i,:,:], time_list[i],status_list[i] = ETO_PLD(N,N_x,nu0, nu, r0, r,c_l,d_r,rev_gap,num_c,Time_Limit);
        if status_list[i] != "OPTIMAL"
            status_all = status_list[i]
            println("Warning: The optimization for test instance $i did not reach optimality. Status: ", status_list[i])
            break
        end
        profit_list[i] = calculate_profit_Sin(theta_true, r0, r, x_list[i,:,:], z_input)                       
    end
    RST_Dict["obj"] = obj_list
    RST_Dict["X"] = x_list
    RST_Dict["time"] = time_list
    RST_Dict["profit"] = profit_list
    RST_Dict["status"] = status_list
    return RST_Dict,status_all
end

solve_ETO_This_Sin (generic function with 1 method)

In [27]:
function solve_RO_this_Sin(S_test,N,N_x,theta_Input,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub)
    r0 = r_params.r0; r = r_params.r;
    RST_Dict = Dict();
    
    obj_list = zeros(S_test); x_list = zeros(S_test,N,N_x); time_list = zeros(S_test); profit_list = zeros(S_test); status_list = Vector{String}(undef, S_test)
    status_all = "OPTIMAL"
    for i in 1:S_test
        z_input = Z_test[i,:];                
        nu0, nu = compute_w(theta_Input,z_input)
        obj_list[i], x_list[i,:,:], time_list[i],status_list[i] = RO_PLD(N,N_x,nu0,nu,r0,r,c_l,d_r,rev_gap,psi_lb,psi_ub,phi_lb,phi_ub,gamma,dual_norm,num_c,Time_Limit)
        if status_list[i] != "OPTIMAL"
            status_all = status_list[i]
            println("Warning: The RO optimization for test instance $i did not reach optimality. Status: ", status_list[i])
            break
        end
        profit_list[i] = calculate_profit_Sin(theta_true, r0, r, x_list[i,:,:], z_input)
    end
    RST_Dict["obj"] = obj_list
    RST_Dict["X"] = x_list
    RST_Dict["time"] = time_list
    RST_Dict["profit"] = profit_list
    RST_Dict["status"] = status_list
    return RST_Dict,status_all
end

solve_RO_this_Sin (generic function with 1 method)

In [28]:
function ETO_Optimization_Process_Sin(S_train_list,lambda_all,instances)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data_Size = load(string(data_dir, "Input_Data_Sin_S=$(S_train).jld2"));

        RST_True_All = Dict()
        RST_ETO_All = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Input_Data_Size["ins=$(ins)"]
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            theta_true = Input_Data_this["theta_true"]
            nu_all_true = Input_Data_this["nu_true"]
            Z_test = Input_Data_this["Z_test"]

            # ******** ETO Model *************
            for lambda in lambda_all
                theta_hat = Input_Data_this["theta_hat_lambda=$lambda"]
                nu_all_hat = Input_Data_this["nu_hat_lambda=$lambda"]
                RST_ETO,status_ETO = solve_ETO_This_Sin(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test)
                println("ins=$(ins)_lambda=$(lambda) profit = ",RST_ETO["profit"][1])
                RST_ETO_All["ins=$(ins)_lambda=$(lambda)"] = RST_ETO
            end
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "RST_ETO_Sin_S=$(S_train).jld2"), RST_ETO_All);
    end
end

ETO_Optimization_Process_Sin (generic function with 1 method)

In [29]:
function RO_Optimization_Process_Sin(S_train_list,lambda_all,instances)
    for S_train in S_train_list
        println("********** S_train = ",S_train," **********")
        Input_Data_Size = load(string(data_dir, "Input_Data_Sin_S=$(S_train).jld2"));

        RST_RO_All = Dict()
        ins = 1
        while ins <= instances
            # ******** Data generation *************
            Input_Data_this = Input_Data_Size["ins=$(ins)"]
            theta_true,r_params,X_train,Y_train,Z_train,Asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
            theta_true = Input_Data_this["theta_true"]
            nu_all_true = Input_Data_this["nu_true"]
            Z_test = Input_Data_this["Z_test"]

            # ******** RO Model *************
            for lambda in lambda_all
                theta_hat = Input_Data_this["theta_hat_lambda=$lambda"]
                nu_all_hat = Input_Data_this["nu_hat_lambda=$lambda"]
                for g_index in 1:length(gamma_list)
                    gamma = gamma_list[g_index]
                    RST_RO,status_RO = solve_RO_this_Sin(S_test,N,N_x,theta_hat,theta_true,r_params,c_l,d_r,rev_gap,num_c,Time_Limit,Z_test,gamma,psi_lb,psi_ub,phi_lb,phi_ub)
                    println("lambda = $lambda, gamma = $gamma, RO profit = ",RST_RO["profit"][1])
                    RST_RO_All["ins=$(ins)_lambda=$(lambda)_gamma=$(gamma)"] = RST_RO
                end
            end
            println("******* ins = ",ins,"*********")
            ins = ins + 1
        end
        save(string(data_dir, "RST_RO_Sin_S=$(S_train).jld2"), RST_RO_All);
    end
end

RO_Optimization_Process_Sin (generic function with 1 method)

In [30]:
Estimation_Process_Sin(S_train_list,lambda_all,instances)

********** S_train = 500 **********
******* ins = 1*********
******* ins = 2*********
******* ins = 3*********
******* ins = 4*********
******* ins = 5*********
******* ins = 6*********
******* ins = 7*********
******* ins = 8*********
******* ins = 9*********
******* ins = 10*********
******* ins = 11*********
******* ins = 12*********
******* ins = 13*********
******* ins = 14*********
******* ins = 15*********
******* ins = 16*********
******* ins = 17*********
******* ins = 18*********
******* ins = 19*********
******* ins = 20*********
******* ins = 21*********
******* ins = 22*********
******* ins = 23*********
******* ins = 24*********
******* ins = 25*********
******* ins = 26*********
******* ins = 27*********
******* ins = 28*********
******* ins = 29*********
******* ins = 30*********
******* ins = 31*********
******* ins = 32*********
******* ins = 33*********
******* ins = 34*********
******* ins = 35*********
******* ins = 36*********
******* ins = 37*********
******* ins

In [31]:
ETO_Optimization_Process_Sin(S_train_list,lambda_all,instances)

********** S_train = 500 **********
ins=1_lambda=0.005 profit = 0.5468426502386566
ins=1_lambda=0.0075 profit = 0.5468426502386566
ins=1_lambda=0.01 profit = 0.5468426502386566
******* ins = 1*********
ins=2_lambda=0.005 profit = 0.5206097597823094
ins=2_lambda=0.0075 profit = 0.518150247965795
ins=2_lambda=0.01 profit = 0.518150247965795
******* ins = 2*********
ins=3_lambda=0.005 profit = 0.4877763918775375
ins=3_lambda=0.0075 profit = 0.4862678794158994
ins=3_lambda=0.01 profit = 0.44952363827829667
******* ins = 3*********
ins=4_lambda=0.005 profit = 0.4955787143625661
ins=4_lambda=0.0075 profit = 0.4955787143625661
ins=4_lambda=0.01 profit = 0.4955787143625661
******* ins = 4*********
ins=5_lambda=0.005 profit = 0.530538774375765
ins=5_lambda=0.0075 profit = 0.530538774375765
ins=5_lambda=0.01 profit = 0.530538774375765
******* ins = 5*********
ins=6_lambda=0.005 profit = 0.4991874747535082
ins=6_lambda=0.0075 profit = 0.4991874747535082
ins=6_lambda=0.01 profit = 0.49918747475350

In [ ]:
RO_Optimization_Process_Sin(S_train_list,lambda_all,instances)

********** S_train = 500 **********
lambda = 0.005, gamma = 0.0, RO profit = 0.5468426502386566
lambda = 0.005, gamma = 0.05, RO profit = 0.5468426502386566
lambda = 0.005, gamma = 0.1, RO profit = 0.5468426502386566
lambda = 0.005, gamma = 0.2, RO profit = 0.549448960371353
lambda = 0.005, gamma = 0.4, RO profit = 0.5389475693763996
lambda = 0.005, gamma = 0.6, RO profit = 0.5297194953285564
lambda = 0.005, gamma = 0.8, RO profit = 0.5297194953285564
lambda = 0.005, gamma = 1.0, RO profit = 0.5297194953285564
lambda = 0.0075, gamma = 0.0, RO profit = 0.5468426502386566
lambda = 0.0075, gamma = 0.05, RO profit = 0.5468426502386566
lambda = 0.0075, gamma = 0.1, RO profit = 0.5468426502386566
lambda = 0.0075, gamma = 0.2, RO profit = 0.5428538923625652
lambda = 0.0075, gamma = 0.4, RO profit = 0.5389475693763996
lambda = 0.0075, gamma = 0.6, RO profit = 0.5297194953285564
lambda = 0.0075, gamma = 0.8, RO profit = 0.5297194953285564
lambda = 0.0075, gamma = 1.0, RO profit = 0.529719495328

In [ ]:
profit_ETO_all = Dict()
for S_train in S_train_list
    RST_ETO_All = load(string(data_dir, "RST_ETO_Sin_S=$(S_train).jld2"))
    for lambda in lambda_all
        profit_ETO_all["S=$(S_train)_lambda=$(lambda)"] = [RST_ETO_All["ins=$(ins)_lambda=$(lambda)"]["profit"][1] for ins in 1:instances];
    end
end
profit_ETO_Avg = [[mean(profit_ETO_all["S=$(S_train)_lambda=$(lambda)"]) for lambda in lambda_all] for S_train in S_train_list];

In [ ]:
# S_index = 1
# S_train = S_train_list[S_index]
# plot(lambda_all, profit_ETO_Avg[S_index]./profit_True_Avg[S_index],marker=:o,label="S=$S_train")
# for S_index in 2:length(S_train_list)
#     S_train = S_train_list[S_index]
#     plot!(lambda_all, profit_ETO_Avg[S_index]./profit_True_Avg[S_index],marker=:o,label="S=$S_train")
# end
# display(current())